# An application of convolution in machine learning

Sample MNIST images:

![MNIST examples](https://www.researchgate.net/profile/Stefan_Elfwing/publication/266205382/figure/fig5/AS:267913563209738@1440886979379/Example-images-of-the-ten-handwritten-digits-in-the-MNIST-training-set.png)

- 10 classes
- 60 thousand training images
- 10 thousand testing images
- Each image is monochrome, 28-by-28 pixels.

In [ ]:
#@title Import the required modules
import torch
import torch.nn as nn
import torchvision.datasets as datasets
import torchvision.transforms as transforms

In [ ]:
# Define the "device". If GPU is available, device is set to use it, otherwise CPU will be used.
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
#@title Download the dataset
train_data = datasets.MNIST(root = './data', train = True,
                        transform = transforms.ToTensor(), download = True)

test_data = datasets.MNIST(root = './data', train = False,
                       transform = transforms.ToTensor())

In [ ]:
target_class = 3

train_data.targets[train_data.targets!=target_class] = 0
train_data.targets[train_data.targets==target_class] = 1

test_data.targets[test_data.targets!=target_class] = 0
test_data.targets[test_data.targets==target_class] = 1

In [ ]:
# About the ToTensor() transformation.

# PyTorch networks expect a tensor as input with dimensions N*C*H*W  where
# N: batch size
# C: channel size
# H: height
# W: width

# Normally an image is of size H*W*C.
# ToTensor() transformation moves the channel dimension to the beginning as needed by PyTorch.

In [ ]:
#@title Define the data loaders
batch_size = 100
train_loader = torch.utils.data.DataLoader(dataset = train_data,
                                             batch_size = batch_size,
                                             shuffle = True)

test_loader = torch.utils.data.DataLoader(dataset =  test_data ,
                                      batch_size = batch_size,
                                      shuffle = False)

In [ ]:
#@title Define a CNN network

class CNN(nn.Module):
    #This defines the structure of the NN.
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 1, kernel_size=28, bias=False)
        #self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        #self.conv2_drop = nn.Dropout2d()  #Dropout
        self.fc1 = nn.Linear(4, 1)
        #self.fc2 = nn.Linear(64, 10)
        self.pool = nn.MaxPool2d(2)
        self.relu = nn.ReLU()
        self.sig = nn.Sigmoid()

    def forward(self, x):
        #print(self.conv1(x).shape)
        x = self.conv1(x)
        #print(x.shape)
        #x = self.pool(self.relu(self.conv2(x)))
        x = self.sig(x.view(-1, 1))
        #print(x.shape)
        #x = self.fc1(x)
        #x = F.dropout(x, training=self.training)
        #x = self.fc2(x)
        return x


# Create an instance
net = CNN().to(device)

In [ ]:
print(net)

In [ ]:
#@title Define the loss function and the optimizer
loss_fun = nn.MSELoss()
#optimizer = torch.optim.SGD( net.parameters(), lr=1.e-3)
optimizer = torch.optim.SGD( net.parameters(), lr=0.001, momentum=.9)

In [ ]:
#@title Train the model
losses = []
iter_nums = []
num_epochs = 7
for epoch in range(num_epochs):
  for i ,(images,labels) in enumerate(train_loader):
    images = images.to(device)
    labels = labels.to(device)
    labels = labels[:,None].float()

    optimizer.zero_grad()
    output = net(images)
    loss = loss_fun(output, labels)
    loss.backward()
    optimizer.step()

    if epoch==4:
      optimizer = torch.optim.SGD( net.parameters(), lr=0.0001, momentum=.9)

    if (i+1) % batch_size == 0:
      losses.append(loss.item())
      iter_nums.append(epoch*len(train_loader)+i)
      print('Epoch [%d/%d], Step [%d/%d], Loss: %.4f'
                 %(epoch+1, num_epochs, i+1, len(train_data)//batch_size, loss.item()))

In [ ]:
import matplotlib.pyplot as plt

plt.plot(iter_nums,losses)
plt.xlabel('iters')
plt.ylabel('loss')
plt.show()

In [ ]:
#@title Run the trained model on the testing set

correct = 0
total = 0
for images,labels in test_loader:
  images = images.to(device)
  labels = labels.to(device)

  out = net(images)
  _, predicted_labels = torch.max(out,1)
  correct += (predicted_labels == labels).sum()
  total += labels.size(0)

print('Percent correct: %.3f %%' %((100*correct)/(total+1)))

In [ ]:
weights = net.conv1.weight.data.clone().cpu().numpy()#  .features[1].weight.data.clone()
print(weights.shape)

In [ ]:
from matplotlib import pyplot as plt
filter = weights.reshape(28,28)
plt.figure(i, figsize=(1.2,1.2))
plt.axis('off')
plt.imshow(filter, cmap='gray')
plt.colorbar()


End of the notebook.

---
Related content: 

[Explore convolution.](in0401)

[Explore convolution of two exponential functions.](in0402)

[Explore cross-correlation and auto-correlation.](in0403)